In [187]:
# Se importan las librerias Necesarias
import pandas as pd
import numpy as np
import geopy
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn import linear_model
geopy.geocoders.options.default_user_agent = "my-application"

#La Etapa nro 1 consta de Seleccionar los datos mas relevantes para nuestro modelo, limpiarlos y rellenarlos con los datos faltantes

In [188]:
#Se importa el Dataset ("properties_colombia_test.csv") a un dataframe utilizando pandas
df = pd.read_csv("properties_colombia_train.csv")

In [189]:
#Utilizando como criterio valor la media de la variable price, categorizamos las propiedades entre baratas (0)
# Y caras (1) creando la columna target 

promedio = df["price"].mean()
df["target"] = [1 if x > promedio else 0 for x in df["price"]]

In [190]:
#Se elimina la columna price dado que ya tenemos categorizadas las propiedas con el target

df= df.drop("price",axis=1)

In [191]:
#Eliminamos otras  columnas que no aportan datos de calidad al modelo o que son redundantes con otras columnas

df = df.drop(["ad_type","operation_type","l1","l4","l5","l6","currency","id","created_on","surface_total","price_period","geometry","start_date","Unnamed: 0"],axis=1)

In [192]:
#Corregimos errores de la columna End_date
df["end_date"] = df["end_date"].replace("9999-12-31", "2020-12-31")

In [193]:
#Completamos los campos faltantes de latitudes y longitudes (lat,lon) utilizando las ubicaciones provistas en las columnas de localizacion 
#Haciendo Uso de la herramiento GeoPY 

geolocator = Nominatim(user_agent="elpepp51@gmail.com")
def completar_Latitud(x):
    if pd.isnull(x["lat"]):
        location1 = geolocator.geocode(str(x["l2"]) + "," + str(x["l3"]))
        latitud = location1.latitude
        return(latitud)
    else:
        return(x["lat"])

def completar_longitud(x):
    if pd.isnull(x["lon"]):
        location1 = geolocator.geocode(str(x["l2"]) + "," + str(x["l3"]))
        longitud = location1.longitude
        return(longitud)
    else:
        return(x["lon"])

def completar_lonlati(columna):
    if pd.isnull(columna):
        resultado = columna.mean()
        return(resultado)
    else:
        return(columna)
        
mean_value = df["lat"].mean()
df["lat"].fillna(value=mean_value,inplace=True)
mean_value2 = df["lon"].mean()
df["lon"].fillna(value=mean_value2,inplace=True)


In [194]:
# Rellenamos Valores Faltantes de las columnas Room, Bedroom y Bathroom

mean_value = df["rooms"].mean()
df["rooms"].fillna(value=mean_value,inplace=True)
mean_value2 = df["bathrooms"].mean()
df["bathrooms"].fillna(value=mean_value2,inplace=True)
mean_value3 = df["bedrooms"].mean()
df["bedrooms"].fillna(value=mean_value3,inplace=True)
df["l3"].fillna(value="otro",inplace=True)




In [195]:
#Rellenamos los valores de surfaced_covered utilizando como referencia la descripcion de la propiedad
formatos = ["mt2","m2","mt","mts","MTS","MT","METROS","M2","MT2","mts2","metros"]
def rellenar_metros(x):
    if pd.isnull(x["surface_covered"]):
        string = x["description"]
        condition = False
        for element in formatos:
               if type(string) != str:
                    break
               else:
                    if string.find(element) != -1:
                     string.find(element)
                     condition = True
                     break
        if condition == True:

            mtrs = string[string.find(element)-3:string.find(element)]
            mtrs = mtrs.replace(" ","")
            if mtrs.isdigit():
                if mtrs == "0.00" or mtrs == "0.0" or 0 or 0.00 or "00.0" or "0":
                    mtrs = 50
                mtrs = int(mtrs)
                return(mtrs)
            else:
                return(x["surface_covered"])
        else:
            return(x["surface_covered"])
    else:
        return(x["surface_covered"])

df["surface_covered"] = df.apply(rellenar_metros,axis=1)

mean_value4 = df["surface_covered"].mean()
df["surface_covered"].fillna(value=mean_value4,inplace=True)

df = df.drop(["title","description"],axis=1)

#En la Etapa Nro 2 normalizamos todas las columnas Codificando todos los No-Numericos a Numericos para que pueda aplicarse el Modelo de ML, Ademas los Datos que ya son numericos son convertidos al mismo Tipo de Dato.

In [196]:
#Utilizando la herramienta Lebel Encoding de la libreria scikit learn, codificamos utilizando etiquetas numericas a los datos no numericos
#Esta es la mejor opcion de codificado en este caso debido al gran numero de datos categoricos que tenemos
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df["end_date"] = le.fit_transform(df["end_date"])
df["l2"] = le.fit_transform(df["l2"])
df["l3"] = le.fit_transform(df["l3"])
df["property_type"] = le.fit_transform(df["property_type"])


In [197]:
#Convertimos Todos los Float a int 

df = df.astype(int)

In [198]:
#Por ultimo cargamos el dataset de Test ("properties_colombia_test.csv") como hicimos con el Train y le realizamos exactamente los mismos procesos y
#modificaciones (exceptuando lo realizado en la columna Price)

df_test = pd.read_csv("properties_colombia_test.csv")

df_test.head()
df_test = df_test.drop(["ad_type","operation_type","l1","l4","l5","l6","currency","id","created_on","surface_total","price_period","geometry","start_date","Unnamed: 0"],axis=1)
df_test["end_date"] = df_test["end_date"].replace("9999-12-31", "2020-12-31")
mean_value = df_test["lat"].mean()
df_test["lat"].fillna(value=mean_value,inplace=True)
mean_value2 = df_test["lon"].mean()
df_test["lon"].fillna(value=mean_value2,inplace=True)
mean_value = df_test["rooms"].mean()
df_test["rooms"].fillna(value=mean_value,inplace=True)
mean_value2 = df_test["bathrooms"].mean()
df_test["bathrooms"].fillna(value=mean_value2,inplace=True)
mean_value3 = df_test["bedrooms"].mean()
df_test["bedrooms"].fillna(value=mean_value3,inplace=True)
df_test["l3"].fillna(value="otro",inplace=True)
df_test["surface_covered"] = df_test.apply(rellenar_metros,axis=1)

mean_value4 = df_test["surface_covered"].mean()
df_test["surface_covered"].fillna(value=mean_value4,inplace=True)

df_test = df_test.drop(["title","description"],axis=1)
le = preprocessing.LabelEncoder()
df_test["end_date"] = le.fit_transform(df_test["end_date"])
df_test["l2"] = le.fit_transform(df_test["l2"])
df_test["l3"] = le.fit_transform(df_test["l3"])
df_test["property_type"] = le.fit_transform(df_test["property_type"])
df_test = df_test.astype(int)




In [199]:
# Le añadimos una columna "target" al dataframe de test para que alli el modelo realice las predicciones
df_test["target"] = ""


In [200]:
#formateamos la columna target
df_test["target"] = df_test["target"].dropna()
df_test["target"] = le.fit_transform(df_test["target"])

En la Parte nro 3 elegimos el modelo de prediccion que consideramos adecuado, y usando los datasets de entramiento y testeo que limpiamos anterior
mente


In [201]:
#Elegimos como modelo de ML a un arbol de decisiones dado que es muy util en situacion de muchas variables categoricas donde 
#El resultado de la prediccione es Binario

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Definimos El modelo y elegimos como variables de entrenamiento  el dataset alojado en la variable "df" y como variables de testeo el alojado en
# "df_test"

model = DecisionTreeClassifier(max_depth=3)
x_train = df.drop(["target"],axis=1)
y_train = df["target"]
x_test = df_test.drop(["target"],axis=1)
y_test = df_test["target"]

#Entrenamos el modelo y realizamos la prediccion

model = model.fit(x_train,y_train)
predicciones = model.predict(x_test)


df_test["target"] = predicciones
df_test["target"].value_counts()


0    57237
1     8613
Name: target, dtype: int64

In [202]:

model.score(x_train,y_train)

0.8160962596621598

In [203]:
#Creamos un Dataframe con la columna de predicciones

pred = pd.DataFrame()
pred["pred"] = prediccion

pred.to_csv("cybrexd.csv",index=False)

print(pred)

       pred
0         0
1         0
2         0
3         0
4         0
...     ...
65845     0
65846     0
65847     0
65848     0
65849     0

[65850 rows x 1 columns]
